In [0]:
#🎯 Goal

#Create consumption-ready tables for:
#Students per class & academic year
#Gender distribution per class
#Parent contact coverage (DQ KPI)
#All outputs → managed Delta tables

In [0]:
silver_df = spark.table("student_project.silver_student_dimension")

In [0]:
silver_df.select("academic_yr").distinct().show()

In [0]:
from pyspark.sql.functions import col
valid_students_df = silver_df.filter(col("is_valid_student") == True)

In [0]:
students_per_class_df = valid_students_df.groupBy(
    "academic_yr",
    "class_id",
    "class_name"
).count() \
 .withColumnRenamed("count", "total_students")

In [0]:
students_per_class_df.show()

In [0]:
students_per_class_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("student_project.gold_students_per_class")

In [0]:
gender_distribution_df = valid_students_df.groupBy(
    "academic_yr",
    "class_name",
    "gender"
).count() \
 .withColumnRenamed("count", "total_students")

In [0]:
gender_distribution_df.show()

In [0]:
gender_distribution_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("student_project.gold_gender_distribution")

In [0]:
from pyspark.sql.functions import sum, when
parent_contact_kpi_df = valid_students_df.groupBy(
    "academic_yr"
).agg(
    sum(when(col("has_parent_contact") == True, 1).otherwise(0))
        .alias("students_with_parent_contact"),
    sum(when(col("has_parent_contact") == False, 1).otherwise(0))
        .alias("students_missing_parent_contact")
)

In [0]:
parent_contact_kpi_df.show()

In [0]:
parent_contact_kpi_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("student_project.gold_parent_contact_kpi")

In [0]:
spark.sql("SHOW TABLES IN student_project").show()